In [1]:
from prj.config import DATA_DIR
from prj.data.data_loader import DataConfig, DataLoader
import polars as pl
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from prj.data.data_loader import PARTITIONS_DATE_INFO
import pandas as pd
import gc
SEED = 42

2024-12-28 18:07:31.015675: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-28 18:07:31.015710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-28 18:07:31.017014: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-28 18:07:31.023711: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-28 18:07:31.816405: W tensorflow/compiler/tf2

In [2]:
from catboost import Pool, CatBoostRegressor

def evaluate_model(model, train_ds: pl.LazyFrame, val_ds: pl.LazyFrame, features, cat_features=[], target='responder_6'):
    if len(cat_features) > 0:
        print(f'Using categorical features: {cat_features}')
    
    X = train_ds.select(features).collect().to_numpy()
    y = train_ds.select(target).collect().to_series().to_numpy()
    w = train_ds.select('weight').collect().to_series().to_numpy()
    
    print(f'Fitting model: {X.shape}')
    model.fit(X, y, verbose=100)
    
    del X, y, w
    gc.collect()
    
    X_val = val_ds.select(features).collect().to_numpy()
    y_val = val_ds.select(target).collect().to_series().to_numpy()
    w_val = val_ds.select('weight').collect().to_series().to_numpy()
    print(f'Evaluating model: {X_val.shape}')

    y_hat_val = model.predict(X_val).clip(-5, 5)
    return r2_score(y_val, y_hat_val, sample_weight=w_val)

In [3]:
start_dt, end_dt = 1100, 1150
start_val_dt = 1140

start_dt, end_dt = PARTITIONS_DATE_INFO[8]['min_date'], PARTITIONS_DATE_INFO[9]['max_date']
start_val_dt = PARTITIONS_DATE_INFO[9]['min_date']

In [4]:
params = {'iterations': 600, 'learning_rate': 0.05, 'task_type': 'GPU', 'has_time': True}

In [5]:
data_args = {}
config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)
complete_ds = loader.load(start_dt, end_dt)
features = loader.features
cat_features = []


train_ds = complete_ds.filter(pl.col('date_id') < start_val_dt)
val_ds = complete_ds.filter(pl.col('date_id') >= start_val_dt)

# to_norm_features = [f'feature_{i:02d}' for i in range(79)]
# global_mean = train_ds.select(to_norm_features).mean().collect().to_dicts()[0]
# global_std = train_ds.select(to_norm_features).std(ddof=0).collect().to_dicts()[0]

# train_ds = train_ds.with_columns(
#     (pl.col(f).sub(global_mean[f])).truediv(global_std[f]) for f in to_norm_features
# )
# val_ds = val_ds.with_columns(
#     (pl.col(f).sub(global_mean[f])).truediv(global_std[f]) for f in to_norm_features
# )
    


model = CatBoostRegressor(**params)

evaluate_model(model, train_ds, val_ds, features, cat_features=cat_features)

Fitting model: (6140024, 79)
0:	learn: 0.8669842	total: 151ms	remaining: 1m 30s
100:	learn: 0.8572267	total: 14.1s	remaining: 1m 9s
200:	learn: 0.8536788	total: 28.4s	remaining: 56.5s
300:	learn: 0.8504632	total: 43s	remaining: 42.7s
400:	learn: 0.8480313	total: 57.4s	remaining: 28.5s
500:	learn: 0.8458133	total: 1m 11s	remaining: 14.2s
599:	learn: 0.8438725	total: 1m 25s	remaining: 0us
Evaluating model: (6274576, 79)


0.006586825075253655

In [6]:
data_args = {'include_time_id': True}

config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)
complete_ds = loader.load(start_dt, end_dt)
features = loader.features
cat_features = []


train_ds = complete_ds.filter(pl.col('date_id') < start_val_dt)
val_ds = complete_ds.filter(pl.col('date_id') >= start_val_dt)


model = CatBoostRegressor(**params)

evaluate_model(model, train_ds, val_ds, features, cat_features=cat_features)

Fitting model: (6140024, 80)
0:	learn: 0.8669842	total: 153ms	remaining: 1m 31s
100:	learn: 0.8572428	total: 15.2s	remaining: 1m 15s
200:	learn: 0.8534901	total: 30.6s	remaining: 1m
300:	learn: 0.8503839	total: 46.1s	remaining: 45.8s
400:	learn: 0.8475703	total: 1m 1s	remaining: 30.5s
500:	learn: 0.8451894	total: 1m 16s	remaining: 15.2s
599:	learn: 0.8432094	total: 1m 32s	remaining: 0us
Evaluating model: (6274576, 80)


0.0071020673539048396

In [7]:
data_args = {'include_time_id': True, 'include_intrastock_norm_temporal': True}

config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)
complete_ds = loader.load(start_dt, end_dt)
features = loader.features
cat_features = []


train_ds = complete_ds.filter(pl.col('date_id') < start_val_dt)
val_ds = complete_ds.filter(pl.col('date_id') >= start_val_dt)


model = CatBoostRegressor(**params)

evaluate_model(model, train_ds, val_ds, features, cat_features=cat_features)

100%|██████████| 346/346 [00:20<00:00, 16.48it/s]


Fitting model: (6140024, 134)
0:	learn: 0.8669484	total: 222ms	remaining: 2m 12s
100:	learn: 0.8566541	total: 19.2s	remaining: 1m 35s
200:	learn: 0.8522820	total: 38s	remaining: 1m 15s
300:	learn: 0.8489371	total: 56.7s	remaining: 56.4s
400:	learn: 0.8462104	total: 1m 15s	remaining: 37.3s
500:	learn: 0.8438359	total: 1m 33s	remaining: 18.5s
599:	learn: 0.8414090	total: 1m 52s	remaining: 0us
Evaluating model: (6274576, 134)


0.007491846912200395

In [8]:
data_args = {'include_time_id': True, 'include_intrastock_norm': True}

config = DataConfig(**data_args)
loader = DataLoader(data_dir=DATA_DIR, config=config)
complete_ds = loader.load(start_dt, end_dt)
features = loader.features
cat_features = []


train_ds = complete_ds.filter(pl.col('date_id') < start_val_dt)
val_ds = complete_ds.filter(pl.col('date_id') >= start_val_dt)


model = CatBoostRegressor(**params)

evaluate_model(model, train_ds, val_ds, features, cat_features=cat_features)

Fitting model: (6140024, 134)
0:	learn: 0.8669557	total: 202ms	remaining: 2m
100:	learn: 0.8563351	total: 19.4s	remaining: 1m 35s
200:	learn: 0.8520242	total: 37.9s	remaining: 1m 15s
300:	learn: 0.8484665	total: 56.6s	remaining: 56.2s
400:	learn: 0.8454909	total: 1m 15s	remaining: 37.3s
500:	learn: 0.8431576	total: 1m 34s	remaining: 18.6s
599:	learn: 0.8409899	total: 1m 52s	remaining: 0us
Evaluating model: (6274576, 134)


0.006364631186449143

In [9]:
pd.DataFrame(model.get_feature_importance(), index=features, columns=['importance']).sort_values(by='importance', ascending=False).style.background_gradient(cmap='viridis')

,importance
feature_24,3.915219
time_id_norm,3.428398
feature_04,3.364799
feature_08,3.321559
feature_75,3.183053
feature_38_mean,3.091352
feature_36,2.726459
feature_58,2.311714
feature_05_mean,2.195576
feature_38,2.113313
